# 作業重點:

(1)以, Adam, 為例, 調整 batch_size, epoch , 觀察accurancy, loss 的變化

(2)以同一模型, 分別驗證 SGD, Adam, Rmsprop 的 accurancy

# 作業目標:
    
    取得各種優化器的運算結果

In [1]:
from __future__ import print_function
import keras
import tensorflow
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import optimizers
import itertools
import pandas as pd 


In [2]:
LEARNING_RATE = 1e-3
num_classes = 10
data_augmentation = True
num_predictions = 20


In [3]:

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# 資料正規化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [5]:
#    第一步：選擇模型, 順序模型是多個網絡層的線性堆疊
 
model = Sequential()

#   第二步：構建網絡層
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense( 10)) # 輸出結果是10個類別，所以維度是10   
model.add(Activation('softmax')) # 最後一層用softmax作為激活函數

2021-12-06 12:00:43.562997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:
2021-12-06 12:00:43.563035: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-06 12:00:43.563055: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu2010): /proc/driver/nvidia/version does not exist
2021-12-06 12:00:43.563346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# 模型建立完成後，統計參數總量
print("Total Parameters：%d" % model.count_params())

Total Parameters：1250858


In [7]:
# 輸出模型摘要資訊
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                        

### (1)以, Adam, 為例, 調整 batch_size, epoch , 觀察accurancy, loss 的變化

### (2)以同一模型, 分別驗證 SGD, Adam, Rmsprop 的 accurancy

In [ ]:
batch_sizes = [128]
epochs = [10, 20, 30]
optimizer_set = [keras.optimizers.gradient_descent_v2.SGD(learning_rate=LEARNING_RATE, nesterov=True, momentum=0.95),
                 tensorflow.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                 tensorflow.keras.optimizers.RMSprop(learning_rate=LEARNING_RATE)]

results = []
for i, (batch_size, epoch, opt) in enumerate(itertools.product(batch_sizes, epochs, optimizer_set)):
    print(f"exp: {i}, batch_size={batch_size}, epochs: {epoch}, optimizer:{type(opt).__name__}")
    
    keras.backend.clear_session()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

        
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    history=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epoch,
              validation_data=(x_test, y_test),
              shuffle=True)       
    
    results.append( {'exp': i,
                     'batch_size': batch_size,
                     'epoch': epoch,
                     'optimizer': type(opt).__name__,
                     'train-loss': model.history.history["loss"],
                     'valid-loss': model.history.history["val_loss"],
                     'train-acc': model.history.history["accuracy"],
                     'valid-acc': model.history.history["val_accuracy"],
                    })    
        

experiments = pd.DataFrame(results, columns =['exp', 'batch_size', 'epoch', 'optimizer', 'train-loss', 'valid-loss', 'train-acc', 'valid-acc'])
experiments

exp: 0, batch_size=128, epochs: 10, optimizer:SGD
Epoch 1/10
391/391 [==============================] - 69s 175ms/step - loss: 2.2134 - accuracy: 0.1628 - val_loss: 1.9958 - val_accuracy: 0.2818
Epoch 2/10
391/391 [==============================] - 66s 169ms/step - loss: 1.9351 - accuracy: 0.2915 - val_loss: 1.7394 - val_accuracy: 0.3821
Epoch 3/10
391/391 [==============================] - 68s 173ms/step - loss: 1.7232 - accuracy: 0.3671 - val_loss: 1.6096 - val_accuracy: 0.4111
Epoch 4/10
 26/391 [>.............................] - ETA: 57s - loss: 1.6691 - accuracy: 0.3864

In [ ]:
from io import StringIO
fout = StringIO('x.x.csv')
experiments.to_csv('./x.csv')

## load model

In [ ]:
from keras.models import load_model
columns = ['loss_score', 'accuracy_score', 'optimizer', 'loss_function', 'epoch', 'learning_rate', 'decacy', \
           'momentum']
loss_accuracy_sore = []

In [ ]:
model_adm1 = load_model(save_dir+"/keras_cifar10_opt_adm_epochs_20.h5")
scores_adm1 = model.evaluate(x_test,y_test,batch_size=200,verbose= 0)
loss_accuracy_sore.append(['', scores_adm1, 'adm', 'categorical_crossentropy', 20, '', '', ''])

In [ ]:
model_RMSprop = load_model(save_dir+"/keras_cifar10_opt_RMSprop_epochs_30_lr_0.01-v1r01.h5")
scores_RMSprop = model.evaluate(x_test,y_test,batch_size=200,verbose= 0)
loss_accuracy_sore.append(['', scores_RMSprop, 'RMSprop', 'categorical_crossentropy', 30, '', '', ''])

In [ ]:
loss_accuracy_sore

In [ ]:
#    第六步：輸出
import numpy 

print ( " test set " )
scores = model.evaluate(x_test,y_test,batch_size=200,verbose= 0)
print ( "" )
#print ( " The test loss is %f " % scores)
print ( " The test loss is %f ", scores)


result = model.predict(x_test,batch_size=200,verbose= 0)

result_max = numpy.argmax(result, axis = 1 )
test_max = numpy.argmax(y_test, axis = 1 )

result_bool = numpy.equal(result_max, test_max)
true_num = numpy.sum(result_bool)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valiidation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valiidation'], loc='upper left')
plt.show()